In [ ]:
%%sh
pip install -qU langchain_community yfinance

In [ ]:
from langchain_community.llms import Ollama
import pprint
import yfinance

In [ ]:
AGENT_MODEL_NAME = "arcee-ai/arcee-agent"

In [ ]:
agent = Ollama(
    model=AGENT_MODEL_NAME,
    system="""You are a helpful chat assistant.""",
    top_p=0.7,
)  # assuming you have run `ollama pull arcee-ai/arcee-agent`

In [ ]:
func_prompt = """You have four primary functions: checking the last price of a specified stock, finding the name of a company CEO, finding what a company does, and answering specific questions about a company. Use the appropriate function based on the user's query.

### Functions:

1. **get_stock_price(company_name: str, stock_symbol: str) -> str**
    - This function returns the last close price of a specified stock.
    - Input: A company name (e.g., Mc Donalds), which you must convert to a stock symbol (e.g., MCD).
    - Output: A string containing the last close price of the specified stock (e.g., "The last closing price of Mc Donalds (MCD) is $250.00").

2. **get_ceo_name(company_name: str, stock_symbol: str) -> str**
- This function returns the name of the CEO of a specified company.
- Input: A company name (e.g., Mc Donalds), which you must convert to a stock symbol (e.g., MCD).
- Output: A string containing the name of the CEO of the specified company (e.g., "The CEO of Mc Donalds is John Doe").

3. **get_company_summary(company_name: str, stock_symbol: str) -> str**
- This function returns a summary describing the business activities of a specified company.
- Input: A company name (e.g., Mc Donalds), which you must convert to a stock symbol (e.g., MCD).
- Output: A string containing a detailed summary of the specified company's business activities.

4. **answer_general_question(question: str) -> str**
    - This function answers questions in general.
    - Input: a user question.
    - Output: your best answer to the user question.

### Instructions:

- If the user asks a question related to the price of a stock, use the `get_stock_price` function.
- If the user asks a question related to the CEO of a company, use the `get_ceo_name` function.
- If the user asks a general question about a company's activities, use the `get_company_summary` function.
- If the user asks any other question, use the `answer_general_question` function. Only return the result of the function call, not your internal reasoning.
- Only return the result of the function call.

### User Query:\n"""

In [ ]:
def function_call(input, model=agent, func_prompt=func_prompt):
    payload = f"""{func_prompt}{input}\n"""
    ans = model.invoke(payload)
    return ans

In [ ]:
def get_stock_price(company_name, stock_symbol):
    stock = yfinance.Ticker(stock_symbol)
    price = stock.history(period="1d")["Close"].values[0]
    return f"The last closing price of {company_name} ({stock_symbol}) was ${price:.2f}."


def get_ceo_name(company_name, stock_symbol):
    stock = yfinance.Ticker(stock_symbol)
    info = stock.info
    ceo = info['companyOfficers'][0]['name']
    return f"The CEO of {company_name} is {ceo}. The full job title is {info['companyOfficers'][0]['title']}."


def get_company_summary(company_name, stock_symbol):
    stock = yfinance.Ticker(stock_symbol)
    summary = stock.info['longBusinessSummary']
    return f"{company_name} ({stock_symbol}) is a company that is involved in {summary}."


def answer_general_question(question):
    ans = agent.invoke(f"""{question}\n""")
    return ans


def llm_pack(input):
    try:
        func = function_call(input).strip()
        print(func)
        code = f"result = {func}"
        local_vars = {}
        exec(code, globals(), local_vars)
        ans = local_vars.get('result').strip()
        return ans
    except Exception as e:
        return

In [ ]:
response = llm_pack("What the stock price for Caterpillar?")
print(response)

In [ ]:
response = llm_pack("Who runs Caterpillar?")
print(response)

In [ ]:
response = llm_pack("What does Caterpillar build?")
pprint.pprint(response)

In [ ]:
response = llm_pack("Who are the main competitors of Caterpillar?")
print(response)